<a href="https://colab.research.google.com/github/helter88/dw_matrix/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install eli5

In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [19]:
cd '/content/drive/My Drive/Colab Notebooks'

/content/drive/My Drive/Colab Notebooks


In [20]:
ls

ColabDriveGithub.ipynb  day4.ipynb  day4_meta.ipynb  day5.ipynb  dw_matrix/


In [21]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix'

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [22]:
ls

data/  dw_matrix/  HelloGithub.ipynb  LICENSE  matrix_one/  README.md


In [23]:
ls data

men_shoes.csv


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[ feats].values
  Y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, Y, scoring= 'neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [26]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [27]:
model = RandomForestRegressor(max_depth =5, n_estimators=100, random_state=0)
run_model(['brand_cat'],model)

(-57.31783843165656, 4.181246596160967)

In [28]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict

  features = literal_eval(x.replace('\\"','"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value
  return output_dict


df['features_parsed']= df['features'].map(parse_features)

In [34]:
keys = set()
df['features_parsed'].map(lambda x: keys.update(x.keys()))

len(keys)

476

In [36]:
def get_name_feat(key):
  return 'feat_' + key
for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [37]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_target audience', 'feat_made in', 'feat_handbag style',
       'feat_charge time (minutes)', 'feat_ground', 'feat_date',
       'feat_contains cubic zirconia', 'feat_retail price', 'feat_spikes type',
       'feat_is wheeled'],
      dtype='object', length=526)

In [0]:
keys_stat = {}
for key in keys:
 keys_stat[key] = df[ False == df[get_name_feat(key)].isnull()].shape[0]/ df.shape[0] * 100

In [39]:
df.shape[0]

18280

In [45]:
{k:v for k,v in keys_stat.items() if v >30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

for key in keys:
  df[get_name_feat(key) +'_cat'] = df[get_name_feat(key)].factorize()[0]
  get_name_feat(key)

In [49]:
df['brand'] = df['brand'].map(lambda x: str(x).lower())

df[ df.brand != df.feat_brand][ ['brand', 'feat_brand']].head()


,brand,feat_brand
21,rubies,generic
22,rubies,generic
23,rubies,generic
24,unbranded,NaN
31,american fighter,NaN


In [0]:
feats = ['']

In [0]:
model = RandomForestRegressor(max_depth = 5, n_estimators = 100)
result = run_model(['brand_cat'],model)

In [0]:
feats_cat = [x for x in df.columns if 'cat' in x]

In [75]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat','feat_style_cat','feat_sport_cat']
#feats += feats_cat

#feats = list(set(feats))

model = RandomForestRegressor(max_depth = 5, n_estimators = 100)
run_model(feats,model)

(-57.07056422606255, 4.2824221566230145)

In [77]:
X = df[feats].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth = 5, n_estimators=100, random_state= 0)
m.fit(X, y)

print(result)
perm = PermutationImportance(m, random_state=1).fit(X,y);
eli5.show_weights(perm, feature_names=feats)

(-57.27945316735154, 4.225127615318966)


Weight,Feature
0.2600 ± 0.0110,brand_cat
0.1047 ± 0.0084,feat_material_cat
0.0458 ± 0.0047,feat_gender_cat
0.0213 ± 0.0011,feat_brand_cat
0.0069 ± 0.0016,feat_style_cat
0.0002 ± 0.0000,feat_sport_cat


In [59]:
df['brand'].value_counts(normalize=True)

nike             0.097210
puma             0.033315
ralph lauren     0.028775
vans             0.021116
new balance      0.020295
                   ...   
marc new york    0.000055
john ashford     0.000055
coggs            0.000055
otz shoes        0.000055
wood n stream    0.000055
Name: brand, Length: 1732, dtype: float64

In [69]:
df[df['brand'] == 'nike'].features_parsed.sample(5).values

array([{'sport': 'running shoes', 'occasion': 'running shoes', 'material': 'synthetic-and-mesh', 'gender': 'men', 'shoe size': '8 us men', 'shoe category': "men's shoes", 'assembled product dimensions (l x w x h)': '12.00 x 8.00 x 6.00 inches', 'fabric content': 'synthetic-and-mesh', 'color': 'black/white', 'model': '554682 010', 'shoe closure': 'lace-up', 'casual & dress shoe style': 'running shoes', 'manufacturer part number': '554682 010', 'variant group id': '224#mp#554682 010', 'brand': 'nike', 'age group': 'men', 'shoe width': 'd(m)'},
       {'condition': 'new with box', 'type': 'cleats'},
       {'style': 'basketball shoes', 'authenticity': '100% authentic guaranteed', 'material': 'synthetic', 'country/region of manufacture': 'china', 'condition': 'new with box'},
       {},
       {'gender': 'men', 'shoe category': 'mens shoes', 'brand': 'nike', 'color': 'pure platinum/hot lava/wolf grey/black', 'casual & dress shoe style': 'athletic sneakers', 'shoe width': 'medium (d, m)'}],

In [70]:
df['feat_age group'].value_counts()

adult               4563
men                  350
child                 77
men's                 33
unisex                 6
infant                 4
toddler                4
mens                   4
boys'                  3
women ,�� unisex       2
women                  2
youth                  2
men||women             2
12 up                  1
adult ,�� teen         1
Name: feat_age group, dtype: int64